### UnarXive Import
Plain text files are loaded, cleaned, combined with metadata, indexed and finally stored in a Document Store for the Haystack integration. 

In [1]:
from arxive_metadata import rocksDB
import json
import re
import os
import unarxive_import_helpers as helpers
from unarxive_import_helpers import textfile_document_saver
from haystack.preprocessor import PreProcessor

05/17/2021 12:14:18 - INFO - faiss.loader -   Loading faiss with AVX2 support.
05/17/2021 12:14:18 - INFO - faiss.loader -   Loading faiss.
/home/daniel/anaconda3/envs/KD/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
05/17/2021 12:14:18 - INFO - farm.modeling.prediction_head -   Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [ ]:
#All unarXive plain text data is saved in "directory"
directory = "/home/daniel/storage/KD/samples"
files = os.listdir(directory)
#Open the directory and list all textfiles
txt_files = list(filter(lambda x: x.endswith(".txt"), files))
#Initialize the database holding the metadata information
url = "http://192.168.178.34:8089"
db = rocksDB.RocksDBAdapter("arXive_metadata", url)
#Process the files. Combine them with their respective metadata and return them as list.
#TODO: should directly put docs into datastore (ElasticSearch)
docs, statistics = helpers.process_documents(directory, txt_files, db, 200)
print("Processed {} documents in {} seconds.\nA total amount of {} documents was found.\nNo metadata was found for {} documents.".format(statistics["no_original_papers"], statistics["elapsed_time"], statistics["no_output_papers"], statistics["no_paper_without_metadata"]))

In [3]:
#All unarXive plain text data is saved in "directory"
directory = "/home/daniel/storage/KD/samples"
files = os.listdir(directory)
#Open the directory and list all textfiles
txt_files = list(filter(lambda x: x.endswith(".txt"), files))
#Initialize the database holding the metadata information
url = "http://192.168.178.34:8089"
db = rocksDB.RocksDBAdapter("arXive_metadata", url)
#Process the files. Combine them with their respective metadata and return them as list.
#TODO: should directly put docs into datastore (ElasticSearch)
processor = PreProcessor(
            clean_empty_lines=True,
            clean_whitespace=True,
            clean_header_footer=True,
            split_by="word",
            split_length=400,
            split_respect_sentence_boundary=True,
            split_overlap=0
        )
document_saver = helpers.textfile_document_saver("/home/daniel/storage/KD/sampledata", processor)
statistics = helpers.process_documents(directory, txt_files[700:750], db, document_saver, 200)
print("Processed {} documents in {} seconds.\nA total amount of {} documents was found.\nNo metadata was found for {} documents.".format(statistics["no_original_papers"], statistics["elapsed_time"], statistics["no_output_papers"], statistics["no_paper_without_metadata"]))

In [ ]:
document_saver = textfile_document_saver("/home/daniel/storage/KD/sampledata")

In [4]:
d = helpers.Document("/home/daniel/storage/KD/samples/0706.0905.txt")

In [ ]:
docs = processor.process(d.get_document_as_dict())

In [ ]:
type(docs[0])

In [6]:
helpers.add_metadata_to_documents([d], db)

In [7]:
d._meta

{'id': '0706.0905',
 'submitter': 'Marek Korkusinski',
 'authors': 'Irene Puerto-Gimenez, Marek Korkusinski, Pawel Hawrylak',
 'title': 'LCHO-CI method for the voltage control of exchange interaction in gated\n  lateral quantum dot networks',
 'journal-ref': None,
 'doi': None,
 'categories': 'cond-mat.mes-hall cond-mat.mtrl-sci',
 'abstract': '  We present a computational LCHO-CI approach allowing for the simulation of\nexchange interactions in gated lateral quantum dot networks. The approach is\nbased on single-particle states calculated using a linear combination of\nharmonic orbitals (LCHO) of each of the dots, and a configuration interaction\n(CI) approach to the interacting electron problem. The LCHO-CI method is\napplied to a network of three quantum dots with one electron spin per dot, and\na Heisenberg spin Hamiltonian is derived. The manipulation of spin states of a\nthree-spin molecule by applying bias to one of the dots is demonstrated and\nrelated to the bias dependence of

In [9]:
import pandas as pd

In [10]:
df = pd.read_json('/home/daniel/storage/KD/paperswithcode/links-between-papers-and-code.json')

In [11]:
conatain_id = df.dropna(subset=['paper_arxiv_id'])

In [ ]:
def _remove_duplicates(l:list):
    return list(dict.fromkeys(l)) 

In [14]:
def combine_with_papers_with_code(meta:dict):
    return conatain_id.loc[conatain_id["paper_arxiv_id"] == meta["id"]]
    matching_id = conatain_id.loc[conatain_id["paper_arxiv_id"] == meta["id"]]
    if matching_id.__len__() > 0:
        meta["pwc_url"] = _remove_duplicates(matching_id["paper_url"].tolist())
        meta["pwc_repo_url"] = _remove_duplicates(matching_id["repo_url"].tolist())
        meta["pwc_framework"] = _remove_duplicates(matching_id["framework"].tolist())

In [15]:
combine_with_papers_with_code(d._meta)

,paper_url,paper_title,paper_arxiv_id,paper_url_abs,paper_url_pdf,repo_url,mentioned_in_paper,mentioned_in_github,framework
